In [5]:
#librerias 
#redes neuronales
import numpy as np
np.random.seed(4)
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM

#modelo arima
from pmdarima import auto_arima

#regresion lineal
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

#importar documentos
from dotenv import load_dotenv
import os
import pymongo
import os
import pandas as pd
from pymongo import MongoClient

#ignorar advertencias python
import warnings
warnings.filterwarnings('ignore')

#medir tiemop
import time






In [6]:
#importacion documentos
# Load environment variables from the .env file
load_dotenv()
# Retrieve the MongoDB connection URI from the environment variables
dbURI = os.getenv("MONGODB_URI")
# Connect to MongoDB using the connection URI
client = pymongo.MongoClient(dbURI)
# Access the "Project" database (it's not physically created until it has content)
db = client['Project']
#creacion datos capacidad
collection = db['capacity']
documents = collection.find()

data = []
for doc in documents:
    province = doc["_id"]
    for energy_type, year_data in doc.items():
        if energy_type != "_id":  # Skip the _id field
            for year, value in year_data.items():
                data.append({
                    "province": province,
                    "energy type": energy_type,
                    "year": int(year),
                    "value": value
                })

# Create a DataFrame
df = pd.DataFrame(data)
#print(df)

#creacion base de datos generacion
collection1 = db['generation']
documents1 = collection1.find()

data1 = []
for doc in documents1:
    province = doc["_id"]
    for energy_type, year_data in doc.items():
        if energy_type != "_id":  # Skip the _id field
            for year, value in year_data.items():
                data1.append({
                    "province": province,
                    "energy type": energy_type,
                    "year": int(year),
                    "value": value
                })

df1 = pd.DataFrame(data1)
#print(df1)

#creacion base de datos capacidad por sector 
collection2 = db['capacity_by_sector']
documents2 = collection2.find()

data2 = []

for document in documents2:
    sector = document["_id"]
    # Iterate over each year in the document
    for year in document:
        if year != "_id" and isinstance(document[year], dict):  # Check if it's a year key and the value is a dictionary
            # Access the subdocument for the year
            year_data = document[year]
            # Append a dictionary with the desired structure to the data list
            data2.append({
                "sector": sector,
                "year": year,
                "UOM": year_data["UOM"],
                "value": year_data["VALUE"],
                "growth_percentage": year_data.get("Grow_percentage", None)  # Using .get() to handle missing data
            })

# Convert the list of dictionaries into a DataFrame
df2 = pd.DataFrame(data2)

##informacion de tabla precios
collection3 = db['prices']
documents3 = collection3.find()
data3 = []

for doc in documents3:
    province = doc["_id"]
    # Iterate over each sector in the document
    for sector, years in doc.items():
        if sector != "_id":  
            # Iterate over each year in the sector
            for year, value in years.items():
                data3.append({
                    "Province": province,
                    "Sector": sector,
                    "year": int(year),
                    "value": value
                })

df3 = pd.DataFrame(data3)

#importacion tabla demanda
collection4 = db['demand']
documents4 = collection4.find()
data4 = []

for doc in documents4:
    province = doc["_id"]
    # Iterate over each sector in the document
    for sector, years in doc.items():
        if sector != "_id":  
            # Iterate over each year in the sector
            for year, value in years.items():
                data4.append({
                    "Province": province,
                    "Sector": sector,
                    "year": int(year),
                    "value": value
                })

df4 = pd.DataFrame(data4)

#tabla de emisiones
collection5 = db['emissions']
documents5 = collection5.find()
data5 = []

for doc in documents5:
    # The _id field here seems to represent a category rather than a province
    category = doc["_id"]
    # Iterate over each year in the document
    for year, value in doc.items():
        if year != "_id":
            data5.append({
                "Category": category,
                "year": int(year),
                "value": value
            })
# Convert the list of dictionaries into a DataFrame
df5 = pd.DataFrame(data5)


In [ ]:
#funcion para calcular rsme con dataframes
def cal_RMSE(val_reales, val_predichos):
    val_reales = val_reales.values
    val_predichos = val_predichos.values

    mse = mean_squared_error(val_reales, val_predichos)

    # Calcula el RMSE
    rmse = np.sqrt(mse)

    return rmse

In [ ]:
#funcion para calcular R2 con dataframes
def cal_R2(val_reales, val_predichos):
    # Asegurarse de que val_reales y val_predichos sean arrays de numpy
    val_reales = val_reales.values
    val_predichos = val_predichos.values

    # Calcular R^2
    r2 = r2_score(val_reales, val_predichos)
    return r2

In [ ]:
#----------------
#RED NEURONAL



def Redneuronal(datas,periodos_validacion):
    #Definicion parametros para ajustar ddatos entrada funcion
    p = periodos_validacion
    time_step = 3
    largo_setvalidacion = p + time_step
    largo_data = len(datas)
    largo_setentrenamiento = largo_data - largo_setvalidacion
    #Definicion set entrenamiento y validacion
    set_entrenamiento = datas.iloc[:largo_setentrenamiento+1]
    set_validacion = datas.iloc[largo_setentrenamiento:largo_data]

    #Normalización del set de entrenamiento
    sc = MinMaxScaler(feature_range=(0,1)) 
    set_entrenamiento_escalado = sc.fit_transform(set_entrenamiento) 

    #Definicion de parametros
    #time_step = 3
    X_train = [] 
    Y_train = []
    m = len(set_entrenamiento_escalado) 

    #sets de entrenamiento
    for i in range(time_step,m):
        X_train.append(set_entrenamiento_escalado[i-time_step:i,0])
        Y_train.append(set_entrenamiento_escalado[i,0])

    #ajustes para el modelo keras
    X_train, Y_train = np.array(X_train), np.array(Y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

    #Creacion red LSTM
    dim_entrada = (X_train.shape[1],1)
    dim_salida = 1
    na = 500

    modelo = Sequential()
    modelo.add(LSTM(units=na, input_shape=dim_entrada))
    modelo.add(Dense(units=dim_salida))

    modelo.compile(optimizer='rmsprop', loss='mse')

    modelo.fit(X_train,Y_train,epochs=200 ,batch_size=400, verbose=0)

    #Validación (predicción del valor de la energia)
    x_test = set_validacion.values
    x_test = sc.transform(x_test)
    X_test = []
    for i in range(time_step,len(x_test)):
        X_test.append(x_test[i-time_step:i,0])

    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))

    if len(X_test) > 0:
        prediccion = modelo.predict(X_test)
        prediccion = sc.inverse_transform(prediccion)
    else:
        print("No hay suficientes datos para generar predicciones.")

    #ajustes del largo
    prediction_length = len(prediccion)
    validation_length = len(set_validacion)
    new_length = validation_length - prediction_length

    adjusted_validation_set = set_validacion.iloc[new_length:].copy()
    index_names = adjusted_validation_set.index

    # Convert prediction to DataFrame and adjust the column name
    prediction_df = pd.DataFrame(prediccion)
    
    #print(result)

    #sacar RSME
    val_RSME = cal_RMSE(adjusted_validation_set,prediction_df)

  
    return val_RSME , cal_R2(adjusted_validation_set,prediction_df)



In [ ]:


#Modelo Arima
def ModeloArima(datas,periodos_validacion):
    #arreglando dataset largo
    set_entrenamiento = datas.head(len(datas)-periodos_validacion)
    set_validacion = datas.tail(periodos_validacion)
    #entrenamiento modelo arima
    modelo = auto_arima(
        set_entrenamiento,
        start_p=1, start_q=1, max_p=12, max_q=12, d=None, # Especifica rangos para p, d, q
         # Prueba 'adf' para encontrar el orden de diferenciación óptimo d, si no se especifica
        seasonal=False,  # Asume estacionalidad con un ciclo anual, ajusta según tus datos
        stepwise=True,
        test='kpss',
        suppress_warnings=True,
        error_action="ignore",
        n_jobs=-1 # Utiliza todos los núcleos disponibles para la búsqueda
    )
    # Hacer un pronóstico para los próximos  años 
    pronostico = modelo.predict(n_periods=periodos_validacion)
 

    #sacar RSME
    val_RSME = cal_RMSE(set_validacion, pronostico)

    return  val_RSME, cal_R2(set_validacion, pronostico)

# ModeloArima(datafiltrada,4)


In [ ]:

def RegresionLineal(datas,periodos_validacion):
    #df3 = pd.DataFrame()

    X = datas.index.values.reshape(-1, 1)    #print(X)
    Y = datas.values  # Variable dependiente

    X_train = X[:-periodos_validacion]
    X_test = X[-periodos_validacion:]
    y_train = Y[:-periodos_validacion]
    y_test = Y[-periodos_validacion:]

    # Entrenamiento del modelo de regresión lineal
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predicción para los años en el conjunto de prueba
    y_predict = model.predict(X_test)
    y_predict =pd.DataFrame(y_predict)
    #sacar RSME
    set_validacion = datas[-periodos_validacion:]
    val_RSME = cal_RMSE(set_validacion, y_predict)

    return  val_RSME , cal_R2(set_validacion, y_predict)



In [ ]:
def calcular_errores_optimizado(data, iterables2D, periodos):
    resultados = []
    #data['year'] = data['year'].astype(int)
    #data['year'] = pd.to_datetime(data['year'], format='%Y')
    
    for i in data[iterables2D[0]].unique():
        
        for j in data[iterables2D[1]].unique():
            # Filtra el DataFrame según las condiciones dadas
            data_filtrada = data[(data[iterables2D[0]] == i) & (data[iterables2D[1]] == j)]

            # Asegúrate de que la filtración no resulte en un DataFrame vacío
            if not data_filtrada.empty:
                # Selecciona solo las columnas de interés
                datafiltrada = data_filtrada[["year","value"]]
                datafiltrada = datafiltrada.set_index("year")
                datafiltrada.index.name = None
    
                # Llama a cada uno de los modelos y recopila sus resultados
                RSME_ARI, R2_ARI = ModeloArima(datafiltrada, periodos)
                RSME_RED, R2_RED = Redneuronal(datafiltrada, periodos)
                RSME_RL, R2_RL = RegresionLineal(datafiltrada, periodos)
                
                # Crea un diccionario con todos los resultados y las etiquetas correspondientes
                resultado_temporal = {
                    iterables2D[0]: i, 
                    iterables2D[1]: j, 
                    'RSME_ARI': RSME_ARI, 
                    'R2_ARI': R2_ARI,
                    'RSME_RED':RSME_RED,
                    'R2_RED':R2_RED,
                    'RSME_RL':RSME_RL,
                    'R2_RL':R2_RL,
                }
                # Añade el diccionario a la lista de resultados
                resultados.append(resultado_temporal)

                

    # Devuelve la lista de resultados
    return resultados



#-----------------
def calcular_errores_optimizadouni(data, iterables2D, periodos):
    resultados = []
    #data['year'] = data['year'].astype(int)
    #data['year'] = pd.to_datetime(data['year'], format='%Y')
    
    for i in data[iterables2D[0]].unique():
        
        data_filtrada = data[(data[iterables2D[0]] == i) ]

        # Asegúrate de que la filtración no resulte en un DataFrame vacío
        if not data_filtrada.empty:
            # Selecciona solo las columnas de interés
            datafiltrada = data_filtrada[["year","value"]]
            datafiltrada = datafiltrada.set_index("year")
            datafiltrada.index.name = None

            # Llama a cada uno de los modelos y recopila sus resultados
            RSME_ARI, R2_ARI = ModeloArima(datafiltrada, periodos)
            RSME_RED, R2_RED = Redneuronal(datafiltrada, periodos)
            RSME_RL, R2_RL = RegresionLineal(datafiltrada, periodos)
            
            # Crea un diccionario con todos los resultados y las etiquetas correspondientes
            resultado_temporal = {
                iterables2D[0]: i,  
                'RSME_ARI': RSME_ARI, 
                'R2_ARI': R2_ARI,
                'RSME_RED':RSME_RED,
                'R2_RED':R2_RED,
                'RSME_RL':RSME_RL,
                'R2_RL':R2_RL,
            }
            # Añade el diccionario a la lista de resultados
            resultados.append(resultado_temporal)

            

    # Devuelve la lista de resultados
    return resultados



In [ ]:

#df_capacity = calcular_errores_optimizado(df,["province","energy type"], 3)

In [ ]:
#df_generation = calcular_errores_optimizado(df1,["province","energy type"], 3)

In [ ]:
#df_prices = calcular_errores_optimizado(df3,["Province","Sector"], 3)

In [ ]:
#df_demand = calcular_errores_optimizado(df4,["Province","Sector"], 3)

In [ ]:
#df_emissions = calcular_errores_optimizadouni(df5,["Category"], 3)


In [ ]:
#df_capacity_by_sector = calcular_errores_optimizadouni(df2,["sector"], 3)


In [ ]:
# df_emissions
# df_emissions = pd.DataFrame(df_emissions)
# print(df_emissions)
# #para emisiones redes neuronales
# df_generation = pd.DataFrame(df_generation)
# print(df_generation)
# #para generacion redes neuronales
# df_capacity_by_sector = pd.DataFrame(df_capacity_by_sector)
# df_capacity_by_sector.groupby("sector").sum()
# df_capacity_by_sector
# #para capacidad por redes neuronales
# df_capacity
# df_capacity = pd.DataFrame(df_capacity)
# print(df_capacity)
# #para capacidad redes neuronales
# df_prices 
# df_prices = pd.DataFrame(df_prices)
# print(df_prices.sum())
# #para precios es arima
# df_demand 
# df_demand = pd.DataFrame(df_demand)
# print(df_demand.sum())
# #para demanda es arima


In [ ]:

#--------------------------------------
# creacion nueva red nuronal

def RedNeuronalValFuturos(datas,periodos_a_predecir,time_step):

    # Normalización de todo el conjunto de datos 'datas'
    sc = MinMaxScaler(feature_range=(0, 1))
    datas_escalado = sc.fit_transform(datas)

    # Preparación de los datos de entrenamiento
    X_train, Y_train = [], []
    for i in range(time_step, len(datas_escalado)):
        X_train.append(datas_escalado[i-time_step:i, 0])
        Y_train.append(datas_escalado[i, 0])

    X_train, Y_train = np.array(X_train), np.array(Y_train)
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

    # Definición y entrenamiento de la red LSTM
    modelo = Sequential([
        LSTM(500, input_shape=(time_step, 1)),
        Dense(1)
    ])
    modelo.compile(optimizer='rmsprop', loss='mse')
    modelo.fit(X_train, Y_train, epochs=200, batch_size=400, verbose=0)

    # Preparación para predicciones futuras
    # Aquí asumimos que quieres generar predicciones continuando desde el final de 'datas'
    ultimo_input = datas_escalado[-time_step:]
    X_pred = np.array([ultimo_input])
    X_pred = np.reshape(X_pred, (X_pred.shape[0], X_pred.shape[1], 1))

    predicciones_futuras = []
    for _ in range(periodos_a_predecir):
        pred_nueva = modelo.predict(X_pred)[0]
        predicciones_futuras.append(pred_nueva)
        
        # Actualiza 'X_pred' para incluir la nueva predicción al final y quitar el valor más antiguo al inicio
        X_pred = np.append(X_pred, [[pred_nueva]], axis=1)
        X_pred = X_pred[:, 1:, :]

    # Inversión de la normalización para obtener valores reales
    predicciones_futuras = sc.inverse_transform(predicciones_futuras)

    # Conversión de las predicciones a un formato más legible/fácil de manejar, como un DataFrame
    predicciones_df = pd.DataFrame(predicciones_futuras, columns=['Predicción'])

    return predicciones_df


#----------------
#aplicacion modelo por tabla
def calcular_pronostico(data, iterables2D,periodos_a_predecir,time_step):
    resultados = []

    for i in data[iterables2D[0]].unique():
        
        for j in data[iterables2D[1]].unique():
            # Filtra el DataFrame según las condiciones dadas
            data_filtrada = data[(data[iterables2D[0]] == i) & (data[iterables2D[1]] == j)]

            # Asegúrate de que la filtración no resulte en un DataFrame vacío
            if not data_filtrada.empty:
                # Selecciona solo las columnas de interés
                datafiltrada = data_filtrada[["year","value"]]
                datafiltrada = datafiltrada.set_index("year")
                datafiltrada.index.name = None
            
            #configurar los index 
            resultado_temporal = {
                iterables2D[0]: i, 
                iterables2D[1]: j, 
                'Pronostico': predicciones_df(datafiltrada,periodos_a_predecir,time_step)
            }


    # Devuelve la lista de resultados
    return resultados







In [ ]:
x=calcular_pronostico(df,["province","energy type"],5,5)
x